# Configure the ray.tune.TuneConfig behavior of NeuralForecast Auto models
> Tutorial on how to modify the [ray.tune.TuneConfig](https://docs.ray.io/en/latest/tune/api/doc/ray.tune.TuneConfig.html) such that we can specify settings such as `time_budget_s`, running Tune experiments with [BOHB](https://docs.ray.io/en/latest/tune/examples/bohb_example.html)

## Load libraries

In [ ]:
import numpy as np

from neuralforecast import NeuralForecast
from neuralforecast.auto import AutoNHITS
from neuralforecast.utils import AirPassengersPanel

from ray import tune
from ray.tune.search.bohb import TuneBOHB
from ray.tune.schedulers import HyperBandForBOHB

from utilsforecast.plotting import plot_series

/root/miniconda3/envs/neuralforecast/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-11 18:23:55,594	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-03-11 18:23:55,782	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


## Data

We use the AirPassengers dataset for the demonstration of conformal prediction.


In [ ]:
AirPassengersPanel_train = AirPassengersPanel[AirPassengersPanel['ds'] < AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test = AirPassengersPanel[AirPassengersPanel['ds'] >= AirPassengersPanel['ds'].values[-12]].reset_index(drop=True)
AirPassengersPanel_test['y'] = np.nan
AirPassengersPanel_test['y_[lag12]'] = np.nan

## Model training

We now train a AutoNHITS model on the above dataset. We consider two different predictions:
1. Training using the default choice of ray.tune.TuneConfig
2. Training using a customized ray.tune.TuneConfig


In [ ]:
horizon = 12
config = {
     "input_size": tune.choice([horizon]),
    "n_freq_downsample": tune.choice(
        [
            [168, 24, 1],
            [24, 12, 1],
            [180, 60, 1],
            [60, 8, 1],
            [40, 20, 1],
            [1, 1, 1],
        ]
    ),
    "max_steps": tune.choice([10]),
}

model = AutoNHITS(h=horizon, config=config, alias='AutoNHITS-1')
nf = NeuralForecast(models=[model], freq='ME')
nf.fit(AirPassengersPanel_train)



2025-03-11 18:25:57,016	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-03-11 18:25:57,038	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/_train_tune_2025-03-11_18-25-42' in 0.0203s.
2025-03-11 18:25:57,051	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/root/ray_results/_train_tune_2025-03-11_18-25-42", trainable=...)
2025-03-11 18:25:57,070	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 8 trial(s):
- _train_tune_fe24d_00002: FileNotFoundError('Could not fetch metrics for _train_tune_fe24d_00002: both result.json and progress.csv were not found at /root/ray_results/_train_tune_2025-03-11_18-25-42/_train_tune_fe24d_00002_2_input_size=12,max_steps=10,n_

Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  2.47it/s, v_num=94, train_loss_step=41.70, train_loss_epoch=41.70]

`Trainer.fit` stopped: `max_steps=10` reached.


Epoch 9: 100%|██████████| 1/1 [00:00<00:00,  2.45it/s, v_num=94, train_loss_step=41.70, train_loss_epoch=41.70]


In [ ]:
horizon = 12
config = {
     "input_size": tune.choice([horizon]),
    "n_freq_downsample": tune.choice(
        [
            [168, 24, 1],
            [24, 12, 1],
            [180, 60, 1],
            [60, 8, 1],
            [40, 20, 1],
            [1, 1, 1],
        ]
    )
}

model = AutoNHITS(h=horizon, config=config, alias='AutoNHITS-1')

# customize the ray.tune.TuneConfig
model2 = AutoNHITS(h=horizon, config=config, alias='AutoNHITS-2')
algo = TuneBOHB()
algo = tune.search.ConcurrencyLimiter(algo, max_concurrent=4)
scheduler = HyperBandForBOHB(
    time_attr="training_iteration",
    max_t=100,
    reduction_factor=4,
    stop_last_trials=False,
)
model2.set_ray_tune_config(
    metric="loss",
    mode="min",
    search_alg=algo,
    scheduler=scheduler,
    num_samples=10,    
)

nf = NeuralForecast(models=[model, model2], freq='ME')
nf.fit(AirPassengersPanel_train)
preds = nf.predict(futr_df=AirPassengersPanel_test)

2025-03-11 18:06:55,835	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2025-03-11 18:06:55,842	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/_train_tune_2025-03-11_18-04-50' in 0.0055s.
2025-03-11 18:06:59,502	WARNING tune.py:1056 -- Experiment has been interrupted, but the most recent state was saved.
Resume experiment with: Tuner.restore(path="/root/ray_results/_train_tune_2025-03-11_18-04-50", trainable=...)
2025-03-11 18:06:59,512	WARNING experiment_analysis.py:180 -- Failed to fetch metrics for 9 trial(s):
- _train_tune_d8b63_00001: FileNotFoundError('Could not fetch metrics for _train_tune_d8b63_00001: both result.json and progress.csv were not found at /root/ray_results/_train_tune_2025-03-11_18-04-50/_train_tune_d8b63_00001_1_input_size=12,n_freq_downsamp

Epoch 50:   0%|          | 0/1 [00:00<?, ?it/s, v_num=92, train_loss_step=23.60, train_loss_epoch=23.60]        


Detected KeyboardInterrupt, attempting graceful shutdown ...


NameError: name 'exit' is not defined

In [ ]:
plot_series(AirPassengersPanel_train, preds)

NameError: name 'preds' is not defined

We can clearly notice the prediction outputs are different due to the change in ray.tune.TuneConfig